In [1]:
from utils import load_model_from_ckpt, build_model
from pathlib import Path
from config import Config
import yaml
from model import FMRIModel
import torch

In [11]:
float(torch.rand(1)) < 0.5

True

In [44]:
checkpoint = "u8816u7l"
name = "name"
if not checkpoint:
    raise ValueError("Please provide a checkpoint to load.")
print(f"Using checkpoint: {checkpoint}")

model_ckpt_path = Path(f"checkpoints/{checkpoint}/final_model.pt")
params_path = Path(f"checkpoints/{checkpoint}/config.yaml")

with params_path.open("r") as f:
    config_dict = yaml.safe_load(f)
    config = Config(**config_dict)
    
feature_paths_new = {name: Path(config.features_dir).absolute() / path for name, path in config.features.items()}

model = build_model(config)

model.load_state_dict(torch.load(model_ckpt_path, weights_only=False, map_location=torch.device('cpu')), strict=True)
model.eval()

Using checkpoint: u8816u7l


FMRIModel(
  (encoder): ModalityFusionTransformer(
    (projections): ModuleDict(
      (aud_last): Sequential(
        (0): Linear(in_features=2560, out_features=256, bias=True)
      )
      (pool_concat): Sequential(
        (0): Linear(in_features=9216, out_features=256, bias=True)
      )
      (slow_res3_act): Sequential(
        (0): Linear(in_features=4096, out_features=256, bias=True)
      )
      (text): Sequential(
        (0): Linear(in_features=3584, out_features=256, bias=True)
      )
      (thinker_36): Sequential(
        (0): Linear(in_features=4096, out_features=256, bias=True)
      )
      (vis_merged): Sequential(
        (0): Linear(in_features=4096, out_features=256, bias=True)
      )
    )
    (subject_embeddings): Embedding(5, 256)
    (run_embeddings): Embedding(3, 256)
    (transformer): TransformerEncoder(
      (layers): ModuleList(
        (0): TransformerEncoderLayer(
          (self_attn): MultiheadAttention(
            (out_proj): NonDynamicallyQuan

In [45]:
feature_paths_old = {
    "aud_last": "features/Omni/Qwen2.5_3B/features_tr1.49_len8_before6/aud_last", #torch.Size([102, 1280])
    "aud_ln_post": "features/Omni/Qwen2.5_3B/features_tr1.49_len8_before6/audio_ln_post", #torch.Size([102, 1280])
    "conv3d_features": "features/Omni/Qwen2.5_3B/features_tr1.49_len8_before6/conv3d_features", #torch.Size([3536, 1280])
    "vis_block5": "features/Omni/Qwen2.5_3B/features_tr1.49_len8_before6/vis_block5", #torch.Size([3536, 1280])
    "vis_block8": "features/Omni/Qwen2.5_3B/features_tr1.49_len8_before6/vis_block8", #torch.Size([3536, 1280])
    "vis_block12": "features/Omni/Qwen2.5_3B/features_tr1.49_len8_before6/vis_block12", #torch.Size([3536, 1280])
    "vis_merged": "features/Omni/Qwen2.5_3B/features_tr1.49_len8_before6/vis_merged", #torch.Size([884, 2048])
    "thinker_12": "features/Omni/Qwen2.5_3B/features_tr1.49_len8_before6/thinker_12", #torch.Size([1, 984, 2048])
    "thinker_24": "features/Omni/Qwen2.5_3B/features_tr1.49_len8_before6/thinker_24", #torch.Size([1, 984, 2048])
    "thinker_36": "features/Omni/Qwen2.5_3B/features_tr1.49_len8_before6/thinker_36", #torch.Size([1, 984, 2048])
    "text": "features/Text/Qwen2.5_7B_Full_tr1.49",
    "fast_res3_act": "features/Visual/SlowFast_R101_tr1.49/fast_res3_act",
    "fast_stem_act": "features/Visual/SlowFast_R101_tr1.49/fast_stem_act",
    "pool_concat": "features/Visual/SlowFast_R101_tr1.49/pool_concat",
    "slow_res3_act": "features/Visual/SlowFast_R101_tr1.49/slow_res3_act",
    "slow_res5_act": "features/Visual/SlowFast_R101_tr1.49/slow_res5_act",
    "slow_stem_act": "features/Visual/SlowFast_R101_tr1.49/slow_stem_act",
    "audio_long_context": "features/Audio/Wave2Vec2/features_chunk1.49_len60_before50",
    # "audio_mfcc_mono": "features/Audio/LowLevel/_chunk1.49_len4.0_before2.0_nmfcc32_nstats4/mono/movies/",
    # "audio_mfcc_stereo": "features/Audio/LowLevel/_chunk1.49_len4.0_before2.0_nmfcc32_nstats4/stereo/movies/",

}

feature_paths_old = {name: Path("/u/danielcs/algonauts/Algonauts-Decoding") / path for name, path in feature_paths_old.items()}

input_dims_old = {
    "aud_last": 1280 * 2,
    #"aud_ln_post": 1280 * 2,
    #"conv3d_features": 1280 * 2,
    #"vis_block5": 1280 * 2,
    #"vis_block8": 1280 * 2,
    #"vis_block12": 1280 * 2,
    "vis_merged": 2048 * 2,
    #"thinker_12": 2048 * 2,
   # "thinker_24": 2048 * 2,
    "thinker_36": 2048 * 2,
    "text": 3584,
    #"fast_res3_act": 2048,
    #"fast_stem_act": 1024,
    "pool_concat": 9216,
    "slow_res3_act": 4096,
    #"slow_res5_act": 4096,
    #"slow_stem_act": 8192,
    #"audio_long_context": 2048,
    # "audio_mfcc_mono":int(4*32),
    # "audio_mfcc_stereo":int(4*32)
}


model_old = FMRIModel(
    input_dims_old, 1000, fusion_hidden_dim=256, fuse_mode="concat", subject_count=4,
)

model_old.load_state_dict(torch.load(f"checkpoints/{checkpoint}/final_model.pt", weights_only=False, map_location="cpu"))
model_old.eval()

FMRIModel(
  (encoder): ModalityFusionTransformer(
    (projections): ModuleDict(
      (aud_last): Sequential(
        (0): Linear(in_features=2560, out_features=256, bias=True)
      )
      (vis_merged): Sequential(
        (0): Linear(in_features=4096, out_features=256, bias=True)
      )
      (thinker_36): Sequential(
        (0): Linear(in_features=4096, out_features=256, bias=True)
      )
      (text): Sequential(
        (0): Linear(in_features=3584, out_features=256, bias=True)
      )
      (pool_concat): Sequential(
        (0): Linear(in_features=9216, out_features=256, bias=True)
      )
      (slow_res3_act): Sequential(
        (0): Linear(in_features=4096, out_features=256, bias=True)
      )
    )
    (subject_embeddings): Embedding(5, 256)
    (run_embeddings): Embedding(3, 256)
    (transformer): TransformerEncoder(
      (layers): ModuleList(
        (0): TransformerEncoderLayer(
          (self_attn): MultiheadAttention(
            (out_proj): NonDynamicallyQuan

In [46]:
feature_paths_new == feature_paths_old

True

In [47]:
model == model_old

False

In [48]:
config.input_dims

{'aud_last': 2560,
 'pool_concat': 9216,
 'slow_res3_act': 4096,
 'text': 3584,
 'thinker_36': 4096,
 'vis_merged': 4096}

In [49]:
input_dims_old

{'aud_last': 2560,
 'vis_merged': 4096,
 'thinker_36': 4096,
 'text': 3584,
 'pool_concat': 9216,
 'slow_res3_act': 4096}